In [8]:
from pandas import pandas as pd
import datetime
import hashlib 
import os
from lxml import etree as et
from itertools import islice
from datetime import datetime
import datetime as dt
import time

In [9]:
def flip_function(df):
    flip_data = df.T
    flip_data.columns=flip_data.iloc[0]
    flip_data.reset_index()
    flip_data=flip_data[1:]
    return flip_data

def information_events_utc(df, start_date_time):
#     '''
#     Parameters:
#     df: dataframe
#         dataframe with a column 'StartDateTimeUTC' that is in yyyy-mm-dd format that you want in utc
#     start_date_time: string
#         the start date time frome the project sheet

#     Returns:
#     A formatted dataframe
#     '''


    date = start_date_time.split('T')[0]
    z_thing = '.000000Z'
    previous_hour = 0

    # loop over the dataframe
    for index, row in df.iterrows():

        # #ignore the first and last row
        # if (index == 0) or (index==len(df)-1):
        #     continue


        # check and see if the time is allready formatted
        start_time = row['StartDateTime']

        try:
            if (len(str(start_time))) > 10 or (str(start_time)=='na'):
                continue
        except:
            continue



        # find the hour of the event
        current_hour = str(row['StartDateTime'])[:2]
        # print(current_hour)

        # if the current hour is less than the previous hour, we know we crossed midnight, so increment the day
        if int(current_hour) < int(previous_hour):
            datetime_obj = datetime.strptime(date, '%Y-%m-%d')

            datetime_obj += dt.timedelta(days=1)
            date = str(datetime_obj).split(' ')[0]

        else:
            pass

        parsed_event_start_time = date+'T'+str(start_time)+z_thing
        # print(parsed_event_time)

        # update hour
        previous_hour = current_hour

        # update df
        df.at[index, 'StartDateTime'] = parsed_event_start_time

    if 'EndDateTime' in df.columns:
        date = start_date_time.split('T')[0]
        z_thing = '.000000Z'
        previous_hour = 0


        for index, row in df.iterrows():

            # #ignore the first and last row
            # if (index == 0) or (index==len(df)-1):
            #     continue


            # check and see if the time is allready formatted
            end_time = row['EndDateTime']

            try:
                if (len(str(end_time))) > 10 or (str(end_time)=='nan'):
                    continue

            except:
                continue

            # find the hour of the event
            current_hour = str(row['EndDateTime'])[:2]
            # print(current_hour)

            # if the current hour is less than the previous hour, we know we crossed midnight
            if int(current_hour) < int(previous_hour):
                datetime_obj = datetime.strptime(date, '%Y-%m-%d')

                datetime_obj += dt.timedelta(days=1)
                date = str(datetime_obj).split(' ')[0]
                # print('adding_date')

            else:
                pass

            parsed_event_end_time = date+'T'+str(end_time)+z_thing
            # print(parsed_event_time)

            # update hour
            previous_hour = current_hour

            # update df
            df.at[index, 'EndDateTime'] = parsed_event_end_time

    return df

In [13]:
direc = 'C:/Users/jmdoherty/OneDrive - University of Massachusetts/Desktop/Mac_ADCP_L0'

xlsx = []
for i in os.listdir(direc):
        if i.endswith('.xlsx'):
            i_split = i.split('.')[0]
            list = xlsx.append(i_split)
        else:
            continue
            
for i in range(len(xlsx)):
    f = direc+'/'+xlsx[i] +'.xlsx'
    sbf_f= direc+'/'+xlsx[i] + '.tar.gz'
    
    project_prep = pd.read_excel(f, sheet_name='Project')
    projectdf= pd.DataFrame(project_prep)
    CollectionMetaData=flip_function(projectdf)
    TransectTypeCodes=pd.read_excel(f, sheet_name='TransectTypeCodes')
    auto_start_date_time= CollectionMetaData.iloc[0,7]
    Events = pd.read_excel(f, sheet_name='Events')
    EventsUTC=information_events_utc(Events, auto_start_date_time)
    
    md5_hash=hashlib.md5()
    file=open(sbf_f,"rb")
    content=file.read()
    md5_hash.update(content)
    digest=md5_hash.hexdigest()
    
    file_size = os.path.getsize(sbf_f)
    
    ti_m=os.path.getmtime(sbf_f)
    m_ti=time.ctime(ti_m)
    t_obj=time.strptime(m_ti)
    T_stamp=time.strftime("%Y-%m-%dT%H:%M:%S.000000Z",t_obj)
    
    current_time= str(datetime.now()).replace(" ","T")+("Z")
    
    root= et.Element('GranuleMetaDataFile')
    
    for index,row in CollectionMetaData.iterrows():
        Column_heading_1 = et.SubElement(root,'Project')
        Column_heading_2 = et.SubElement(root, 'FileIdentifier')
    
    root_tags=et.SubElement(root,'CollectionMetaData')
    
    for index,row in CollectionMetaData.iterrows():
        Column_heading_3 = et.SubElement(root_tags, 'Campaign')
        Column_heading_4 = et.SubElement(root_tags, 'CampaignShortName')
        Column_heading_5 = et.SubElement(root_tags, 'ShortName')
        Column_heading_6 = et.SubElement(root_tags, 'InstrumentName')
        Column_heading_7 = et.SubElement(root_tags, 'Description')

        Column_heading_1.text = str(row['Project'])
        Column_heading_2.text = str(row['FileIdentifier'])
        Column_heading_3.text = str(row['Campaign'])
        Column_heading_4.text = str(row['CampaignShortName'])
        Column_heading_5.text = str(row['ShortName'])
        Column_heading_6.text = str(row['InstrumentName'])
        Column_heading_7.text = str(row['Description'])
        
    root_tags = et.SubElement(root, 'Extent')
    root_tags1 = et.SubElement(root_tags,'TemporalExtent')
    root_tags2 = et.SubElement(root_tags,'SpatialExtent')
    
    for index,row in CollectionMetaData.iterrows():
        Column_heading_8 = et.SubElement(root_tags1, 'StartDateTime')
        Column_heading_9 = et.SubElement(root_tags1, 'EndDateTime')
        Column_heading_10 = et.SubElement(root_tags2, 'WestBoundLongitude')
        Column_heading_11 = et.SubElement(root_tags2, 'EastBoundLongitude')
        Column_heading_12 = et.SubElement(root_tags2, 'SouthBoundLatitude')
        Column_heading_13 = et.SubElement(root_tags2, 'NorthBoundLatitude')

        Column_heading_8.text = str(row['StartDateTime'])
        Column_heading_9.text = str(row['EndDateTime'])
        Column_heading_10.text = str(row['WestBoundLongitude'])
        Column_heading_11.text = str(row['EastBoundLongitude'])
        Column_heading_12.text = str(row['SouthBoundLatitude'])
        Column_heading_13.text = str(row['NorthBoundLatitude'])
        
    root_tags = et.SubElement(root, 'DataGranuleMembers')
    
    for index,row in CollectionMetaData.iterrows():
        root_tags1= et.SubElement(root_tags,'DataGranuleMember')
        Column_heading_14 = et.SubElement(root_tags1, 'LocalGranuleID')
        Column_heading_15 = et.SubElement(root_tags1, 'ProductionDateTime')

        Column_heading_14.text = str(row['LocalGranuleID'])
        Column_heading_15.text = str(current_time)
        
    for index,row in CollectionMetaData.iterrows():
        root_tags1= et.SubElement(root_tags,'DataGranuleMember')
        Column_heading_16 = et.SubElement(root_tags1, 'LocalGranuleID')
        Column_heading_17= et.SubElement(root_tags1, 'ProductionDateTime')
        Column_heading_18 = et.SubElement(root_tags1, 'Checksum')
        Column_heading_19 = et.SubElement(root_tags1, 'ChecksumType')
        Column_heading_20 = et.SubElement(root_tags1, 'Size')

        Column_heading_16.text = str(row['LocalGranuleID'])
        Column_heading_17.text = str(T_stamp)
        Column_heading_18.text = str(digest)
        Column_heading_19.text = str(row['ChecksumType'])
        Column_heading_20.text = str(file_size)
        
    root_tags = et.SubElement(root, 'Additional')
    
    for index,row in CollectionMetaData.iterrows():
        Column_heading_21 = et.SubElement(root_tags, 'Personnel')
        Column_heading_22 = et.SubElement(root_tags, 'Contact')
        Column_heading_23 = et.SubElement(root_tags, 'ADCPModel')
        Column_heading_24 = et.SubElement(root_tags, 'ADCPSerial')
        Column_heading_25 = et.SubElement(root_tags, 'AntennaModel')
        Column_heading_26 = et.SubElement(root_tags, 'IGSCalibration')
        Column_heading_27 = et.SubElement(root_tags, 'Radome')
        Column_heading_28 = et.SubElement(root_tags, 'AntennaSerial')
        Column_heading_29 = et.SubElement(root_tags, 'TransducerDepth_m')
        Column_heading_30 = et.SubElement(root_tags, 'MagneticVariation')
        Column_heading_31 = et.SubElement(root_tags, 'Positionmethod')
        Column_heading_32 = et.SubElement(root_tags, 'TransectLocation')

        Column_heading_21.text = str(row['Personnel'])
        Column_heading_22.text = str(row['Contact'])
        Column_heading_23.text = str(row['ADCPModel'])
        Column_heading_24.text = str(row['ADCPSerial'])
        Column_heading_25.text = str(row['AntennaModel'])
        Column_heading_26.text = str(row['IGSCalibration'])
        Column_heading_27.text = str(row['Radome'])
        Column_heading_28.text = str(row['AntennaSerial'])
        Column_heading_29.text = str(row['TransducerDepth_m'])
        Column_heading_30.text = str(row['MagneticVariation'])
        Column_heading_31.text = str(row['Positionmethod'])
        Column_heading_32.text = str(row['TransectLocation'])
        
    root_tags2=et.SubElement(root_tags,'TransectTypes')
    
    for index, row in TransectTypeCodes.iterrows():
        root_tags3=et.SubElement(root_tags2,'TransectType')
        Column_heading_33 = et.SubElement(root_tags3,'TransectTypeCode')
        Column_heading_34 = et.SubElement(root_tags3,'Description')

        Column_heading_33.text= str(row['TransectTypeCode'])
        Column_heading_34.text= str(row['Description'])
        
    root_tags1=et.SubElement(root_tags,'Events')
    
    for index,row in EventsUTC.iterrows():
        root_tags2 = et.SubElement(root_tags1, 'Event')

        Column_heading_35 = et.SubElement(root_tags2, 'StartDateTime')
        Column_heading_36 = et.SubElement(root_tags2, 'TransectFile')
        Column_heading_37 = et.SubElement(root_tags2, 'TransectStart')
        Column_heading_38 = et.SubElement(root_tags2, 'TransectEnd')
        Column_heading_39 = et.SubElement(root_tags2, 'TransectType')

        Column_heading_35.text = str(row['StartDateTime'])
        Column_heading_36.text = str(row['TransectFile'])
        Column_heading_37.text = str(row['TransectStart'])
        Column_heading_38.text = str(row['TransectEnd'])
        Column_heading_39.text = str(row['TransectType'])
    
    tree = et.ElementTree(root) 
    et.indent(tree, space="\t", level=0)
    tree.write(direc+'/'+xlsx[i]+".xml", encoding="utf-8")

In [ ]:
project_prep = pd.read_excel(f, sheet_name='Project')
projectdf= pd.DataFrame(project_prep)
CollectionMetaData=flip_function(projectdf)
TransectTypeCodes=pd.read_excel(f, sheet_name='TransectTypeCodes')
Events = pd.read_excel(f, sheet_name='Events')
EventsUTC=information_events_utc(Events,"2022-07-27T19:17:01.000000Z")

In [14]:
md5_hash=hashlib.md5()
file=open(svf_f,"rb")
content=file.read()
md5_hash.update(content)
digest=md5_hash.hexdigest()

FileNotFoundError: [Errno 2] No such file or directory: '\\Users\\jmdoherty\\Downloads\\SWOTCalVal_WM_GNSS_L0_Rec2_20220731T162900_20220731T210100.22_'

In [ ]:
file_size = os.path.getsize(svf_f)

In [ ]:
root= et.Element('GranuleMetaDataFile')

In [ ]:
for index,row in CollectionMetaData.iterrows():
    Column_heading_1 = et.SubElement(root,'Project')
    Column_heading_2 = et.SubElement(root, 'FileIdentifier')

In [ ]:
root_tags=et.SubElement(root,'CollectionMetaData')

In [ ]:
for index,row in CollectionMetaData.iterrows():
    Column_heading_3 = et.SubElement(root_tags, 'Campaign')
    Column_heading_4 = et.SubElement(root_tags, 'CampaignShortName')
    Column_heading_5 = et.SubElement(root_tags, 'ShortName')
    Column_heading_6 = et.SubElement(root_tags, 'InstrumentName')
    Column_heading_7 = et.SubElement(root_tags, 'Description')
    
    Column_heading_1.text = str(row['Project'])
    Column_heading_2.text = str(row['FileIdentifier'])
    Column_heading_3.text = str(row['Campaign'])
    Column_heading_4.text = str(row['CampaignShortName'])
    Column_heading_5.text = str(row['ShortName'])
    Column_heading_6.text = str(row['InstrumentName'])
    Column_heading_7.text = str(row['Description'])

In [ ]:
root_tags = et.SubElement(root, 'Extent')
root_tags1 = et.SubElement(root_tags,'TemporalExtent')
root_tags2 = et.SubElement(root_tags,'SpatialExtent')

In [ ]:
for index,row in CollectionMetaData.iterrows():
    Column_heading_8 = et.SubElement(root_tags1, 'StartDateTime')
    Column_heading_9 = et.SubElement(root_tags1, 'EndDateTime')
    Column_heading_10 = et.SubElement(root_tags2, 'WestBoundLongitude')
    Column_heading_11 = et.SubElement(root_tags2, 'EastBoundLongitude')
    Column_heading_12 = et.SubElement(root_tags2, 'SouthBoundLatitude')
    Column_heading_13 = et.SubElement(root_tags2, 'NorthBoundLatitude')

    Column_heading_8.text = str(row['StartDateTime'])
    Column_heading_9.text = str(row['EndDateTime'])
    Column_heading_10.text = str(row['WestBoundLongitude'])
    Column_heading_11.text = str(row['EastBoundLongitude'])
    Column_heading_12.text = str(row['SouthBoundLatitude'])
    Column_heading_13.text = str(row['NorthBoundLatitude'])

In [ ]:
root_tags = et.SubElement(root, 'DataGranuleMembers')

In [ ]:
for index,row in CollectionMetaData.iterrows():
    root_tags1= et.SubElement(root_tags,'DataGranuleMember')
    Column_heading_14 = et.SubElement(root_tags1, 'LocalGranuleID')
    Column_heading_15 = et.SubElement(root_tags1, 'ProductionDateTime')

    Column_heading_14.text = str(row['LocalGranuleID'])
    Column_heading_15.text = str(row['ProductionDateTime'])

In [ ]:
for index,row in CollectionMetaData.iterrows():
    root_tags1= et.SubElement(root_tags,'DataGranuleMember')
    Column_heading_16 = et.SubElement(root_tags1, 'LocalGranuleID')
    Column_heading_17 = et.SubElement(root_tags1, 'Checksum')
    Column_heading_18 = et.SubElement(root_tags1, 'ChecksumType')
    Column_heading_19 = et.SubElement(root_tags1, 'Size')
    
    Column_heading_16.text = str(row['LocalGranuleID_SBF'])
    Column_heading_17.text = str(digest)
    Column_heading_18.text = str(row['ChecksumType'])
    Column_heading_19.text = str(file_size)

In [ ]:
root_tags = et.SubElement(root, 'Additional')

In [ ]:
for index,row in CollectionMetaData.iterrows():
    Column_heading_20 = et.SubElement(root_tags, 'Personnel')
    Column_heading_21 = et.SubElement(root_tags, 'Contact')
    Column_heading_22 = et.SubElement(root_tags, 'ADCPModel')
    Column_heading_23 = et.SubElement(root_tags, 'ADCPSerial')
    Column_heading_24 = et.SubElement(root_tags, 'AntennaModel')
    Column_heading_25 = et.SubElement(root_tags, 'IGSCalibration')
    Column_heading_26 = et.SubElement(root_tags, 'Radome')
    Column_heading_27 = et.SubElement(root_tags, 'AntennaSerial')
    Column_heading_28 = et.SubElement(root_tags, 'TransducerDepth_m')
    Column_heading_29 = et.SubElement(root_tags, 'MagneticVariation')
    Column_heading_30 = et.SubElement(root_tags, 'Positionmethod')
    Column_heading_31 = et.SubElement(root_tags, 'TransectLocation')

    Column_heading_20.text = str(row['Personnel'])
    Column_heading_21.text = str(row['Contact'])
    Column_heading_22.text = str(row['ReceiverModel'])
    Column_heading_23.text = str(row['ReceiverSerial'])
    Column_heading_24.text = str(row['AntennaModel'])
    Column_heading_25.text = str(row['IGSCalibration'])
    Column_heading_26.text = str(row['Radome'])
    Column_heading_27.text = str(row['AntennaSerial'])
    Column_heading_28.text = str(row['TransducerDepth_m'])
    Column_heading_29.text = str(row['MagneticVariation'])
    Column_heading_30.text = str(row['Positionmethod'])
    Column_heading_31.text = str(row['TransectLocation'])

In [ ]:
root_tags2=et.SubElement(root_tags,'TransectTypes')

In [ ]:
for index, row in SurfaceTypeCodes.iterrows():
    root_tags3=et.SubElement(root_tags2,'TransectType')
    Column_heading_32 = et.SubElement(root_tags3,'Code')
    Column_heading_33 = et.SubElement(root_tags3,'Description')
    
    Column_heading_32.text= str(row['Code'])
    Column_heading_33.text= str(row['Description'])

In [ ]:
root_tags1=et.SubElement(root_tags,'Events')

In [ ]:
for index,row in EventsUTC.iterrows():
    root_tags2 = et.SubElement(root_tags1, 'Event')

    Column_heading_34 = et.SubElement(root_tags2, 'StartDateTime')
    Column_heading_35 = et.SubElement(root_tags2, 'TransectFile')
    Column_heading_36 = et.SubElement(root_tags2, 'TransectStart')
    Column_heading_37 = et.SubElement(root_tags2, 'TransectEnd')
    Column_heading_38 = et.SubElement(root_tags2, 'TransectType')

    Column_heading_34.text = str(row['StartDateTime'])
    Column_heading_35.text = str(row['TransectFile'])
    Column_heading_36.text = str(row['TransectStart'])
    Column_heading_37.text = str(row['TransectEnd'])
    Column_heading_38.text = str(row['TransectType'])